# Task 8 of Semeval 2020: Memotion analysis
## Models training and evaluation
This task is divided into 3 subtasks which are detailed below

In [ ]:
import pandas as pd
from joblib import dump, load
from sklearn.linear_model import LogisticRegressionCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from src.models.ordinal_regression import OrdinalClassifier
from src.utils.files import load_dfs
from src.utils.embeddings import retrieve_all_embeds
from src.utils.reports import generate_report

In [3]:
model_path = "data/models/custom"

In [4]:
def evaluate(clf, embeds, y_train, y_dev, filename, multitask=False):
    res = {}
    for item, (X_train, X_dev, X_test) in embeds.items():
        jobfile = "{}/{}_{}.joblib".format(model_path, filename, "_".join(item.split()))
        print("############### Embeddings: {} ####################".format(item))
        if type(clf) == str:
            load(jobfile)
        else:
            clf.fit(X_train, y_train)
            dump(clf, jobfile) 
        y_pred_dev = clf.predict(X_dev)
        y_pred_test = clf.predict(X_test)
        if not multitask:
            rep = classification_report(y_dev, y_pred_dev)
            print(rep)
        else:
            rep = [classification_report(y_dev[:,col], y_pred_dev[:,col]) for col in range(y_dev.shape[1])]
            cols = ["Humour", "Sarcasm", "Offense", "Motivation"]
            for c, r in list(zip(cols, rep)):
                print("results for class {}:\n{}".format(c, r))
        res[item] = {"pred_cls_dev": y_pred_dev, "report_str": rep, "pred_cls_test": y_pred_test}
    return res

In [5]:
df_train, df_dev = load_dfs(["data/train_cleaned_final.csv", "data/dev_cleaned_final.csv"])
embed = retrieve_all_embeds([("data/features/use.pkl.train", "data/features/xception.pkl.train"), 
                             ("data/features/use.pkl.dev","data/features/xception.pkl.dev"),
                             ("data/features/use.pkl.test", "data/features/xception.pkl.test")])

## Task A: sentiment polarity detection
Classify memes as negative, neutral or positive. More details here: https://competitions.codalab.org/competitions/20629
We compare the results of Ordinal classifier with logistic regression, SVM and random forest.
To investigate how each modality contributes to the detection, we test these models with embeddings of sentences only, 
images only and both concatenated.

In [5]:
y_train_a = df_train["Overall_sentiment"].cat.codes
y_dev_a = df_dev["Overall_sentiment"].cat.codes

In [6]:
y_dev_a

0      2
1      0
2      1
3      2
4      1
      ..
995    2
996    2
997    2
998    2
999    2
Length: 1000, dtype: int8

In [7]:
lr_oc = OrdinalClassifier(LogisticRegressionCV(cv=5, random_state=0, solver="saga", max_iter=10000, n_jobs=6))
res_a_lr = evaluate(lr_oc, embed, y_train_a, y_dev_a, "task_a_lr")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        80
           1       0.00      0.00      0.00       302
           2       0.62      1.00      0.76       618

    accuracy                           0.62      1000
   macro avg       0.21      0.33      0.25      1000
weighted avg       0.38      0.62      0.47      1000

############### Embeddings: text only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        80
           1       0.00      0.00      0.00       302
           2       0.62      1.00      0.76       618

    accuracy                           0.62      1000
   macro avg       0.21      0.33      0.25      1000
weighted avg       0.38      0.62      0.47      1000

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        80
           1       0.00      0.00      0.00       302
           2       0.62      1.00      0.76       618

    accuracy                           0.62      1000
   macro avg       0.21      0.33      0.25      1000
weighted avg       0.38      0.62      0.47      1000



/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
knn_oc = OrdinalClassifier(KNeighborsClassifier(n_jobs=6))
res_a_knn = evaluate(knn_oc, embed, y_train_a, y_dev_a, "task_a_knn")

############### Embeddings: image only ####################
              precision    recall  f1-score   support

           0       0.04      0.03      0.03        80
           1       0.30      0.30      0.30       302
           2       0.61      0.64      0.63       618

    accuracy                           0.49      1000
   macro avg       0.32      0.32      0.32      1000
weighted avg       0.47      0.49      0.48      1000

############### Embeddings: text only ####################
              precision    recall  f1-score   support

           0       0.24      0.16      0.19        80
           1       0.47      0.40      0.43       302
           2       0.70      0.77      0.73       618

    accuracy                           0.61      1000
   macro avg       0.47      0.45      0.45      1000
weighted avg       0.59      0.61      0.60      1000

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

  

In [9]:
gnb_oc = OrdinalClassifier(GaussianNB())
res_a_gnb = evaluate(gnb_oc, embed, y_train_a, y_dev_a, "task_a_gnb")

############### Embeddings: image only ####################
              precision    recall  f1-score   support

           0       0.08      0.65      0.14        80
           1       0.27      0.15      0.19       302
           2       0.62      0.16      0.26       618

    accuracy                           0.20      1000
   macro avg       0.32      0.32      0.20      1000
weighted avg       0.47      0.20      0.23      1000

############### Embeddings: text only ####################
              precision    recall  f1-score   support

           0       0.17      0.14      0.15        80
           1       0.40      0.23      0.29       302
           2       0.65      0.80      0.72       618

    accuracy                           0.58      1000
   macro avg       0.41      0.39      0.39      1000
weighted avg       0.54      0.58      0.54      1000

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

  

In [10]:
abc_oc = OrdinalClassifier(AdaBoostClassifier(n_estimators=100, random_state=0))
res_a_abc = evaluate(abc_oc, embed, y_train_a, y_dev_a, "task_a_abc")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.11      0.03      0.04        80
           1       0.00      0.00      0.00       302
           2       0.62      0.98      0.76       618

    accuracy                           0.61      1000
   macro avg       0.24      0.34      0.27      1000
weighted avg       0.39      0.61      0.47      1000

############### Embeddings: text only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.57      0.10      0.17        80
           1       0.00      0.00      0.00       302
           2       0.63      1.00      0.77       618

    accuracy                           0.62      1000
   macro avg       0.40      0.37      0.31      1000
weighted avg       0.43      0.62      0.49      1000

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

           0       0.36      0.05      0.09        80
           1       0.00      0.00      0.00       302
           2       0.62      1.00      0.77       618

    accuracy                           0.62      1000
   macro avg       0.33      0.35      0.28      1000
weighted avg       0.41      0.62      0.48      1000



/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
rf_oc = OrdinalClassifier(RandomForestClassifier(random_state=0, oob_score=True, n_jobs=6))
res_a_rf = evaluate(rf_oc, embed, y_train_a, y_dev_a, "task_a_rf")

############### Embeddings: image only ####################
              precision    recall  f1-score   support

           0       0.07      0.07      0.07        80
           1       0.30      0.25      0.27       302
           2       0.63      0.68      0.65       618

    accuracy                           0.50      1000
   macro avg       0.33      0.33      0.33      1000
weighted avg       0.48      0.50      0.49      1000

############### Embeddings: text only ####################
              precision    recall  f1-score   support

           0       0.95      0.76      0.85        80
           1       0.98      0.76      0.86       302
           2       0.87      0.99      0.93       618

    accuracy                           0.90      1000
   macro avg       0.93      0.84      0.88      1000
weighted avg       0.91      0.90      0.90      1000

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

  

In [12]:
mlp_oc = OrdinalClassifier(MLPClassifier(max_iter=1000))
res_a_mlp = evaluate(mlp_oc, embed, y_train_a, y_dev_a, "task_a_mlp")

############### Embeddings: image only ####################
              precision    recall  f1-score   support

           0       0.09      0.09      0.09        80
           1       0.29      0.27      0.28       302
           2       0.62      0.64      0.63       618

    accuracy                           0.48      1000
   macro avg       0.33      0.33      0.33      1000
weighted avg       0.48      0.48      0.48      1000

############### Embeddings: text only ####################
              precision    recall  f1-score   support

           0       0.92      0.76      0.84        80
           1       0.83      0.82      0.82       302
           2       0.90      0.92      0.91       618

    accuracy                           0.88      1000
   macro avg       0.88      0.83      0.86      1000
weighted avg       0.88      0.88      0.88      1000

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

  

## Task B: Multilabel sentiment detection
Classify memes as Humourous, sarcastics, offensive and/or motivationnal. One meme can have multiple sentiments.
More details here: https://competitions.codalab.org/competitions/20629
We compare the results of OneVsRest classifier with logistic regression, SVM and random forest.
To investigate how each modality contributes to the detection, we test these models with embeddings of sentences only, 
images only and both concatenated.

In [6]:
y_train_b = df_train[["Humour_bin", "Sarcasm_bin", "Offense_bin", "Motivation_bin"]].to_numpy().astype(int)
y_dev_b = df_dev[["Humour_bin", "Sarcasm_bin", "Offense_bin", "Motivation_bin"]].to_numpy().astype(int)

In [7]:
y_dev_b

array([[1, 1, 1, 0],
       [1, 1, 0, 1],
       [0, 0, 0, 0],
       ...,
       [1, 1, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 0]])

In [8]:
lr_ovc = OneVsRestClassifier(LogisticRegressionCV(cv=5, random_state=0, solver="saga", max_iter=10000, n_jobs=6))
res_b_lr = evaluate(lr_ovc, embed, y_train_b, y_dev_b, "task_b_lr")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.77      1.00      0.87       773
           1       0.75      1.00      0.86       751
           2       0.60      1.00      0.75       601
           3       0.00      0.00      0.00       366

   micro avg       0.71      0.85      0.77      2491
   macro avg       0.53      0.75      0.62      2491
weighted avg       0.61      0.85      0.71      2491
 samples avg       0.71      0.80      0.72      2491

############### Embeddings: text only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.77      1.00      0.87       773
           1       0.75      1.00      0.86       751
           2       0.60      1.00      0.75       601
           3       0.00      0.00      0.00       366

   micro avg       0.71      0.85      0.77      2491
   macro avg       0.53      0.75      0.62      2491
weighted avg       0.61      0.85      0.71      2491
 samples avg       0.71      0.80      0.72      2491

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

           0       0.77      1.00      0.87       773
           1       0.75      1.00      0.86       751
           2       0.60      1.00      0.75       601
           3       0.00      0.00      0.00       366

   micro avg       0.71      0.85      0.77      2491
   macro avg       0.53      0.75      0.62      2491
weighted avg       0.61      0.85      0.71      2491
 samples avg   

/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
knn_ovc = OneVsRestClassifier(KNeighborsClassifier(n_jobs=6))
res_b_knn = evaluate(knn_ovc, embed, y_train_b, y_dev_b, "task_b_knn")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.77      0.90      0.83       773
           1       0.76      0.93      0.84       751
           2       0.62      0.70      0.66       601
           3       0.32      0.22      0.26       366

   micro avg       0.69      0.76      0.72      2491
   macro avg       0.62      0.69      0.65      2491
weighted avg       0.66      0.76      0.71      2491
 samples avg       0.69      0.71      0.66      2491

############### Embeddings: text only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.80      0.95      0.86       773
           1       0.78      0.96      0.86       751
           2       0.69      0.80      0.74       601
           3       0.56      0.34      0.43       366

   micro avg       0.74      0.83      0.78      2491
   macro avg       0.71      0.76      0.72      2491
weighted avg       0.73      0.83      0.77      2491
 samples avg       0.74      0.78      0.72      2491

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

           0       0.80      0.95      0.87       773
           1       0.78      0.97      0.87       751
           2       0.69      0.82      0.75       601
           3       0.58      0.34      0.43       366

   micro avg       0.75      0.84      0.79      2491
   macro avg       0.71      0.77      0.73      2491
weighted avg       0.74      0.84      0.78      2491
 samples avg   

/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
gnb_ovc = OneVsRestClassifier(GaussianNB())
res_b_gnb = evaluate(gnb_ovc, embed, y_train_b, y_dev_b, "task_b_gnb")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

              precision    recall  f1-score   support

           0       0.79      0.23      0.36       773
           1       0.73      0.24      0.36       751
           2       0.61      0.19      0.29       601
           3       0.38      0.80      0.51       366

   micro avg       0.53      0.31      0.39      2491
   macro avg       0.63      0.37      0.38      2491
weighted avg       0.67      0.31      0.37      2491
 samples avg       0.45      0.28      0.31      2491

############### Embeddings: text only ####################
              precision    recall  f1-score   support

           0       0.79      0.85      0.82       773
           1       0.77      0.80      0.79       751
           2       0.66      0.66      0.66       601
           3       0.44      0.41      0.42       366

   micro avg       0.71      0.73      0.72      2491
   macro avg       0.66      0.68      0.67      2491
weighted avg       0.70      0.73      0.71      2491
 samples avg      

/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
abc_ovc = OneVsRestClassifier(AdaBoostClassifier(n_estimators=100, random_state=0))
res_b_abc = evaluate(abc_ovc, embed, y_train_b, y_dev_b, "task_b_abc")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.77      0.95      0.85       773
           1       0.75      0.97      0.84       751
           2       0.60      0.80      0.69       601
           3       0.32      0.11      0.17       366

   micro avg       0.70      0.80      0.74      2491
   macro avg       0.61      0.71      0.64      2491
weighted avg       0.66      0.80      0.71      2491
 samples avg       0.70      0.75      0.69      2491

############### Embeddings: text only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.79      0.96      0.87       773
           1       0.77      0.97      0.86       751
           2       0.66      0.87      0.75       601
           3       0.60      0.28      0.39       366

   micro avg       0.74      0.84      0.79      2491
   macro avg       0.71      0.77      0.72      2491
weighted avg       0.73      0.84      0.77      2491
 samples avg       0.73      0.79      0.72      2491

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

           0       0.78      0.95      0.86       773
           1       0.76      0.96      0.85       751
           2       0.65      0.83      0.72       601
           3       0.49      0.25      0.34       366

   micro avg       0.72      0.82      0.77      2491
   macro avg       0.67      0.75      0.69      2491
weighted avg       0.70      0.82      0.75      2491
 samples avg   

/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
rf_ovc = OneVsRestClassifier(RandomForestClassifier(random_state=0,oob_score=True, n_jobs=6))
res_b_rf = evaluate(rf_ovc, embed, y_train_b, y_dev_b, "task_b_rf")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.76      0.80      0.78       773
           1       0.76      0.85      0.80       751
           2       0.61      0.68      0.64       601
           3       0.37      0.30      0.33       366

   micro avg       0.68      0.71      0.69      2491
   macro avg       0.62      0.66      0.64      2491
weighted avg       0.67      0.71      0.69      2491
 samples avg       0.65      0.67      0.61      2491

############### Embeddings: text only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      0.99      0.97       773
           1       0.93      0.99      0.96       751
           2       0.87      0.98      0.92       601
           3       0.98      0.80      0.88       366

   micro avg       0.92      0.96      0.94      2491
   macro avg       0.93      0.94      0.93      2491
weighted avg       0.93      0.96      0.94      2491
 samples avg       0.87      0.89      0.87      2491

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

           0       0.78      1.00      0.87       773
           1       0.75      1.00      0.86       751
           2       0.63      0.94      0.75       601
           3       0.68      0.08      0.14       366

   micro avg       0.72      0.85      0.78      2491
   macro avg       0.71      0.75      0.66      2491
weighted avg       0.72      0.85      0.73      2491
 samples avg   

/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
mlp_ovc = OneVsRestClassifier(MLPClassifier(max_iter=1000))
res_b_mlp = evaluate(mlp_ovc, embed, y_train_b, y_dev_b, "task_b_mlp")

############### Embeddings: image only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precisi

              precision    recall  f1-score   support

           0       0.77      0.79      0.78       773
           1       0.76      0.84      0.80       751
           2       0.61      0.70      0.65       601
           3       0.37      0.31      0.33       366

   micro avg       0.68      0.71      0.69      2491
   macro avg       0.63      0.66      0.64      2491
weighted avg       0.67      0.71      0.69      2491
 samples avg       0.65      0.67      0.62      2491

############### Embeddings: text only ####################


/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      0.96      0.96       773
           1       0.93      0.97      0.95       751
           2       0.90      0.91      0.90       601
           3       0.87      0.84      0.86       366

   micro avg       0.92      0.93      0.93      2491
   macro avg       0.91      0.92      0.92      2491
weighted avg       0.92      0.93      0.93      2491
 samples avg       0.86      0.87      0.85      2491

############### Embeddings: concatenated ####################
              precision    recall  f1-score   support

           0       0.89      0.92      0.91       773
           1       0.88      0.94      0.91       751
           2       0.83      0.84      0.83       601
           3       0.74      0.72      0.73       366

   micro avg       0.85      0.88      0.86      2491
   macro avg       0.83      0.86      0.84      2491
weighted avg       0.85      0.88      0.86      2491
 samples avg   

/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lb732/Projects/memotion_analysis/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Task C: Multilabel sentiment intensity detection
Classify the degree of humour, sarcasm, offense and motivation of each meme. 
One meme can have multiple sentiments of different intensities. Each sentiment intensity is ranked from 0 (not at all) 
to 5 (very much).
More details here: https://competitions.codalab.org/competitions/20629
We compare the results of OneVsRest Ordinal classifier with logistic regression, SVM and random forest.
To investigate how each modality contributes to the detection, we test these models with embeddings of sentences only, 
images only and both concatenated.

In [14]:
cols = ["Humour", "Sarcasm", "Offense", "Motivation"]
y_train_c = pd.concat([df_train[name].cat.codes for name in cols], axis=1).to_numpy()
y_dev_c = pd.concat([df_dev[name].cat.codes for name in cols], axis=1).to_numpy()

In [15]:
y_dev_c

array([[2, 1, 3, 0],
       [1, 1, 0, 1],
       [0, 0, 0, 0],
       ...,
       [1, 1, 2, 0],
       [2, 2, 2, 0],
       [1, 1, 1, 0]], dtype=int8)

In [ ]:
lr_ovc_oc = MultiOutputClassifier(OrdinalClassifier(LogisticRegressionCV(cv=5, random_state=0, solver="saga", max_iter=10000, n_jobs=6)))
res_c_lr = evaluate(lr_ovc_oc, embed, y_train_c, y_dev_c, "task_c_lr", multitask=True)

############### Embeddings: image only ####################


In [ ]:
task_a = res_a_lr["text only"]["pred_cls_test"]
task_b = res_b_lr["text only"]["pred_cls_test"]
task_c = res_c_lr["text only"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_lr.zip")

In [ ]:
knn_ovc_oc = MultiOutputClassifier(OrdinalClassifier(KNeighborsClassifier(n_jobs=6)))
res_c_knn = evaluate(knn_ovc_oc, embed, y_train_c, y_dev_c, "task_c_knn", multitask=True)

In [ ]:
task_a = res_a_knn["text only"]["pred_cls_test"]
task_b = res_b_knn["text only"]["pred_cls_test"]
task_c = res_c_knn["text only"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_knn.zip")

In [ ]:
task_a = res_a_knn["concatenated"]["pred_cls_test"]
task_b = res_b_knn["concatenated"]["pred_cls_test"]
task_c = res_c_knn["concatenated"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_knn_concat.zip")

In [ ]:
gnb_ovc_oc = MultiOutputClassifier(OrdinalClassifier(GaussianNB()))
res_c_gnb = evaluate(gnb_ovc_oc, embed, y_train_c, y_dev_c, "task_c_gnb", multitask=True)

In [ ]:
task_a = res_a_gnb["text only"]["pred_cls_test"]
task_b = res_b_gnb["text only"]["pred_cls_test"]
task_c = res_c_gnb["text only"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_gnb.zip")

In [ ]:
abc_ovc_oc = MultiOutputClassifier(OrdinalClassifier(AdaBoostClassifier(n_estimators=100, random_state=0)))
res_c_abc = evaluate(abc_ovc_oc, embed, y_train_c, y_dev_c, "task_c_abc", multitask=True)

In [ ]:
task_a = res_a_abc["text only"]["pred_cls_test"]
task_b = res_b_abc["text only"]["pred_cls_test"]
task_c = res_c_abc["text only"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_abc.zip")

In [ ]:
rf_ovc_oc = MultiOutputClassifier(OrdinalClassifier(RandomForestClassifier(random_state=0, oob_score=True, n_jobs=6)))
res_c_rf = evaluate(rf_ovc_oc, embed, y_train_c, y_dev_c, "task_c_rf", multitask=True)

In [ ]:
task_a = res_a_rf["text only"]["pred_cls_test"]
task_b = res_b_rf["text only"]["pred_cls_test"]
task_c = res_c_rf["text only"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_rf.zip")

In [ ]:
mlp_ovc_oc = MultiOutputClassifier(OrdinalClassifier(MLPClassifier(max_iter=1000)))
res_c_mlp = evaluate(mlp_ovc_oc, embed, y_train_c, y_dev_c, "task_c_mlp", multitask=True)

In [ ]:
task_a = res_a_mlp["text only"]["pred_cls_test"]
task_b = res_b_mlp["text only"]["pred_cls_test"]
task_c = res_c_mlp["text only"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_mlp.zip")

In [ ]:
task_a = res_a_mlp["concatenated"]["pred_cls_test"]
task_b = res_b_mlp["concatenated"]["pred_cls_test"]
task_c = res_c_mlp["concatenated"]["pred_cls_test"]
r = generate_report(task_a, task_b, task_c, zipname="res_mlp_concat.zip")

## Generating report
We take the results of the best classifier for each task, here the random forest and generate a report following the guidelines provided here: https://competitions.codalab.org/competitions/20629#learn_the_details-submission-guidelines

In [ ]:
task_a = res_a_rf["text only"]["pred_cls_test"]
task_b = res_b_rf["text only"]["pred_cls_test"]
task_c = res_c_rf["text only"]["pred_cls_test"]

In [ ]:
r = generate_report(task_a, task_b, task_c, zipname="res_rf.zip")
print(r)

In [ ]:
task_c

In [ ]:
task_a.shape